In [2]:
import sys
import time
import datetime

In [3]:
TOPIC_Step2_NAME="Sahamyab-Tweets2"
KAFKA_SERVER="kafka-broker:29092"

In [4]:
import os
# https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html

# setup arguments
os.environ['PYSPARK_SUBMIT_ARGS']='--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Step2_7-Memory-Sink") \
    .config("spark.executor.memory", "1024mb") \
    .config("spark.executor.cores","1") \
    .config("spark.cores.max", "1") \
    .config("spark.sql.session.timeZone", "Asia/Tehran") \
    .getOrCreate()    
    
    

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-12fd304e-9d13-4f2e-883f-4facb9778158;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 810ms :: artifacts dl 23ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central in [default]


In [5]:
spark.sparkContext.setLogLevel("ERROR")


In [6]:
# https://sparkbyexamples.com/spark/spark-sql-structtype-on-dataframe/
schema = StructType([StructField("id", StringType(), True),\
                         StructField("content", StringType(), True),\
                         StructField("sendTime", StringType(), True),\
                         StructField("sendTimePersian", StringType(), True),\
                         StructField("senderName", StringType(), True),\
                         StructField("senderUsername", StringType(), True),\
                         StructField("type", StringType(), True),\
                         StructField("hashtags", ArrayType(StringType()), True)
                    ])

In [7]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_SERVER) \
  .option("subscribe", TOPIC_Step2_NAME) \
  .option("startingOffsets", "earliest") \
  .option("kafka.group.id", "step2_7-Memory-Sink")\
  .load()

In [8]:
tweetsStringDF = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

tweetsDF = tweetsStringDF.select(from_json(col("value"), schema).alias("data")).select("data.*")
tweetsDF = tweetsDF.withColumn("timestamp", unix_timestamp("sendTime", "yyyy-MM-dd'T'HH:mm:ss'Z'").cast('timestamp')) \
            .withColumn("persian_timestamp", from_utc_timestamp("timestamp", "Asia/Tehran").cast('timestamp')) \
            .withColumn("persianYear", tweetsDF['sendTimePersian'].substr(0, 4)) \
            .withColumn("persianMonth", tweetsDF['sendTimePersian'].substr(6, 2)) \
            .withColumn("persianDay", tweetsDF['sendTimePersian'].substr(9, 2))


In [9]:
user_activity_count = tweetsDF\
    .withWatermark("persian_timestamp", "10 minutes") \
    .groupBy(
        window(col("persian_timestamp"), "1 hours", "1 hours"), # 10 minute window, updating every 10 minutes
        col("senderUsername"))\
    .count()

In [10]:
# Create query stream with memory sink
queryStream = user_activity_count\
 .writeStream\
 .format("memory")\
 .queryName("user_activity")\
 .outputMode("update")\
 .start()

In [ ]:

from time import sleep
cnt=1
while True : 
    print("-=-"*20)
    print(f"Round :#{cnt:4}")
    cnt+=1
    top10_last_housr_df = spark.sql(
        """
            select
                window.start
                ,window.end
                ,senderUsername
                ,sum(count) as count
            from
                user_activity
            where
                window.start = (select max(window.start) from user_activity)
            group by
                window.start
                ,window.end
                ,senderUsername
            order by
                4 desc
            limit 10
        """
    )
    top10_last_housr_df.show()
    sleep(20)


-=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=-
Round :#   1


+-----+---+--------------+-----+
|start|end|senderUsername|count|
+-----+---+--------------+-----+
+-----+---+--------------+-----+



-=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=-
Round :#   2


+-------------------+-------------------+--------------+-----+
|              start|                end|senderUsername|count|
+-------------------+-------------------+--------------+-----+
|2021-10-08 22:30:00|2021-10-08 23:30:00|      abo09906|  222|
|2021-10-08 22:30:00|2021-10-08 23:30:00|     measam313|  111|
|2021-10-08 22:30:00|2021-10-08 23:30:00|      alis3394|  111|
|2021-10-08 22:30:00|2021-10-08 23:30:00|        agooly|  111|
|2021-10-08 22:30:00|2021-10-08 23:30:00|mohammad_17799|  111|
|2021-10-08 22:30:00|2021-10-08 23:30:00|      amir2489|  111|
|2021-10-08 22:30:00|2021-10-08 23:30:00|     matin2930|  111|
|2021-10-08 22:30:00|2021-10-08 23:30:00|  tavakolizade|  111|
+-------------------+-------------------+--------------+-----+



-=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=-
Round :#   3


+-------------------+-------------------+--------------+-----+
|              start|                end|senderUsername|count|
+-------------------+-------------------+--------------+-----+
|2021-10-08 22:30:00|2021-10-08 23:30:00|      abo09906|  692|
|2021-10-08 22:30:00|2021-10-08 23:30:00|     measam313|  346|
|2021-10-08 22:30:00|2021-10-08 23:30:00|      alis3394|  346|
|2021-10-08 22:30:00|2021-10-08 23:30:00|        agooly|  346|
|2021-10-08 22:30:00|2021-10-08 23:30:00|mohammad_17799|  346|
|2021-10-08 22:30:00|2021-10-08 23:30:00|      amir2489|  346|
|2021-10-08 22:30:00|2021-10-08 23:30:00|     matin2930|  346|
|2021-10-08 22:30:00|2021-10-08 23:30:00|  tavakolizade|  346|
+-------------------+-------------------+--------------+-----+



-=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=--=-
Round :#   4


### Submit Sample Spark App in Pyspark Container Bash 

- Go to Pyspark Shell :
```bash
docker exec -it pyspark bash
```
- cd /opt/spark-app/
- run this command :
```bash
unset PYSPARK_DRIVER_PYTHON
spark-submit --master  spark-master:7077  --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 memory_sink.py
export PYSPARK_DRIVER_PYTHON=python
```